# AES Plaintext Recovery (ResNet)
In this experiment, the residal network tries to guess the plaintext from the ciphertext, helped with ascii per-byte correction. 

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import AESDatasetCiphertextPlaintext
from pipeline import *

2023-05-10 11:26:07.577856: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Importing the dataset

In [2]:
data = AESDatasetCiphertextPlaintext(128, 'small')

train_labels, train_samples, test_labels, test_samples = data.get_data()

In [3]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (189568, 128)
===== Label Shape: (128,)
===== Training Samples Shape: (189568, 128)
===== Sample Shape: (128,)
===== Testing Labels Shape: (81243, 128)
===== Testing Samples Shape: (81243, 128)


## Creating the model

In [4]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [5]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = dim*8
print("Number of units per hidden layer: {}".format(units))

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
# 0.1 to 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=10000,
    decay_rate=0.01)
optimizer = Adam(learning_rate=lr_schedule)
metrics = ['accuracy', 'binary_accuracy']
epochs = 2
batch_size = 5000

Number of units per hidden layer: 1024


### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [6]:

inputs = Input(shape=input_shape)
net = inputs

for _ in range(1):
    net = residual_block(net, units)
    
net = Dense(units=dim, activation='softmax')(net)

neural_network = Model(inputs, net)

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 128)         512         ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 re_lu (ReLU)                   (None, 128)          0           ['batch_normalization[0][0]']    
                                                                                                  
 dense (Dense)                  (None, 1024)         132096      ['re_lu[0][0]']              

### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [14]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/2
38/38 [==============================] - 18s 478ms/step - loss: 0.4339 - accuracy: 0.0000e+00 - binary_accuracy: 0.5661
Epoch 2/2
38/38 [==============================] - 18s 486ms/step - loss: 0.4339 - accuracy: 0.0000e+00 - binary_accuracy: 0.5661


### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [15]:
results, pred = test_model(neural_network, test_samples, test_labels, batch_size, ascii_correction=True)

17/17 [==============================] - 3s 145ms/step


In [17]:
print("Correct bytes: " + str(results["correct_bytes"]))
print("Byte accuracy: " + str(results["byte_accuracy"]))

print("Correct predictions: " + str(results["correct_predictions"]))
print("Accuracy: " + str(results["accuracy"]))

Correct bytes: 9
Byte accuracy: 6.923673424171929e-06
Correct predictions: 0
Accuracy: 0.0
